## Install required packages

In [ ]:
%pip install ydata-profiling

## Imports

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

from ydata_profiling import ProfileReport

from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split, ParameterGrid

from sklearn.metrics import root_mean_squared_error

import xgboost as xgb

# Exploratory Data Analysis

In [ ]:
df = pd.read_csv('train.csv')

In [ ]:
df.head()

In [ ]:
report = ProfileReport(df)

In [ ]:
report.to_notebook_iframe()

In [ ]:
for c in df.columns:
  if df[c].nunique() == len(df):
    print(c, df[c].dtype)

There are 2 columns that have high correlation with each other. These can cause multicolinearity issues, so we should treat them carefully, because they can influence convergence of some training algorithms. But since there are only 2 such features and we use boosting algorithm, we can leave them, since they still can have explanatory power and these types of algorithms handle such features better than e.g. linear regression.


There are also considerable amount of variable that have unique values including target. Normally, this could raise suspisions, since it can be some kind of ID that ended up in feature set accidentally. ID like columns don't carry any information that is useful for modelling, so in this case they should be discarded. However, here all of such columns are real and hence are not ID, so they can carry some information and should not be discarded.


All the distributions look uniform and don't raise any suspicions.
Correlations with the target are all pretty weak.

In [ ]:
df.dtypes

All columns are numeric. There is one boolean column, but it's probably fine to treat it as numeric.

## Correlations

### Numeric vs Numeric

In [ ]:
def plot_numerical_correlation_heatmap(d, method='pearson'):
  # Compute the correlation matrix
  corr = d.corr(method=method)

  # Generate a mask for the upper triangle
  mask = np.triu(np.ones_like(corr, dtype=bool))

  # Set up the matplotlib figure
  f, ax = plt.subplots(figsize=(10, 8))

  # Generate a custom diverging colormap
  cmap = sns.diverging_palette(230, 20, as_cmap=True)

  # Draw the heatmap with the mask and correct aspect ratio
  sns.heatmap(corr, mask=mask, cmap=cmap,
              square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
plot_numerical_correlation_heatmap(df)

In [ ]:
plot_numerical_correlation_heatmap(df, method='spearman')

In [ ]:
df['8'].unique()

There is only one pair of variables that have high correlation. Some models can suffer issues with convergence when there is multicollinearity present. We can try to remove one of these features. Since one of these features is boolean, I'd probably remove this one, because it carries less information than second feature that is a real variable.

# Modelling

In [ ]:
X = df.drop(columns=['target'])
y = df['target']

Split dataset into train/val.

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=7)

In [ ]:
def modelfit(alg, dtrain, target, useTrainCV=True, cv_folds=5, early_stopping_rounds=50):

    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain.values, label=target.values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics=['rmse'], early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])

        print(cvresult)

    alg.fit(dtrain, target)

    #Predict training set:
    dtrain_predictions = alg.predict(dtrain)

    #Print model report:
    print("\nModel Report")
    print("Train RMSE : %.4g" % root_mean_squared_error(target.values, dtrain_predictions))

    if useTrainCV:
      print("Cross Validation RMSE: ", min(cvresult['test-rmse-mean']))

In [ ]:
def grid_search_validation(X, y, X_val, y_val, model, param_grid, score_func=root_mean_squared_error):
  params, test_score = [], []
  for param in param_grid:
    model.set_params(**param)
    model.fit(X, y)

    pred = model.predict(X_val)
    score = score_func(y_val, pred)

    params.append(param)
    test_score.append(score)

  return {'test_score': np.array(test_score),
          'best_score': min(test_score),
          'params': params,
          'best_params': params[np.argmin(test_score)]}

In [ ]:
def plot_hyperparameter_results(cv_results, figsize=(25, 7)):
  test_score = cv_results['test_score']
  X = np.arange(len(test_score))

  _, ax = plt.subplots(figsize=figsize)

  min_ind = np.argmin(test_score)

  ax.set_title(f"Best value:{np.round(test_score[min_ind], decimals=3)}")
  ax.set_xlabel("Parameter values")
  ax.set_ylabel("Score")
  ax.set_xticks(X)

  labels = [str(x) for x in cv_results['params']]
  ax.set_xticklabels(labels, rotation=45, ha='right')

  [t.set_color('red')for i, t in enumerate(ax.xaxis.get_ticklabels()) if i == min_ind ]

  ax.grid()
  ax.plot(
      X, test_score, "o-", color="blue", label="CV Metric value"
  )

  ax.plot(min_ind, test_score[min_ind], 'ro')
  ax.legend(loc="best")

## XGBoost

In [ ]:
xgb1 = xgb.XGBRegressor(
 learning_rate=0.1,
 n_estimators=1000,
 nthread=4,
 seed=27)

X_train_c, y_train_c, X_val_c, y_val_c = X_train.copy(deep=True), \
y_train.copy(deep=True), X_val.copy(deep=True), y_val.copy(deep=True),\


modelfit(xgb1, X_train_c, target=y_train_c)

Let's add some regularization to prevent possible overfit

In [ ]:
xgb2 = xgb.XGBRegressor(
 learning_rate=0.1,
 n_estimators=1000,
 reg_lambda=1e1,
 nthread=4,
 seed=27)

X_train_c, y_train_c, X_val_c, y_val_c = X_train.copy(deep=True), \
y_train.copy(deep=True), X_val.copy(deep=True), y_val.copy(deep=True),\


modelfit(xgb2, X_train_c, target=y_train_c)

### Hyperparameter tuning

Strategy here is to tune small groups of parameters (typically 1-2 parameters at once) from most important to least important ones. Sometimes we will tune one group more than once to explore space near previous optimal value in more detail.

In [ ]:
param_test = {
 'max_depth':range(1,10,2),
 'min_child_weight':range(1,10,2),
}

res = grid_search_validation(X_train, y_train,
                             X_val, y_val, xgb.XGBRegressor(learning_rate=0.1, n_estimators=200, reg_lambda=1e2, seed=27),
                                   ParameterGrid(param_test))

In [ ]:
plot_hyperparameter_results(res, figsize=(35, 7))

In [ ]:
param_test = {
 'n_estimators': [200, 500, 700, 1000],
 'learning_rate': [0.01, 0.1]
}

res = grid_search_validation(X_train, y_train,
                             X_val, y_val, xgb.XGBRegressor(reg_lambda=1e2, max_depth=9, min_child_weight=7, seed=27),
                                   ParameterGrid(param_test))

In [ ]:
plot_hyperparameter_results(res, figsize=(35, 7))

These results suggest that xgboost is a good choice in this scenario. Now we can train it on the whole training dataset with parameters we have identified during this EDA. We didn't try many combinations, but current results are firstly good enough and secondly pretty stable, so other values will probably produce similar validation metric values.